In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout

# **Augmentation function**

In [25]:
datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range= 0.1,
)

In [14]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0


# **Reshape the data fitting for VGG16**

In [ ]:
# Reshape the data into 4D arrays for input to CNN
x_train = x_train.reshape((x_train.shape[0], 28, 28))
x_test = x_test.reshape((x_test.shape[0], 28, 28))
print(x_train.input_shape)
# Convert Data to 3 Channel
x_train=np.stack((x_train,)*3, axis=-1)
x_test=np.stack((x_test,)*3, axis=-1)
print(x_train.input_shape)

In [ ]:
# Convert the labels to categorical one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# **Creating Augmented Data for Training and Testing**

In [31]:
datagen.fit(x_train.reshape((-1,28,28,3)))
augmented_train = datagen.flow(x_train.reshape((-1,28,28,3)),y_train,batch_size=32)

datagen.fit(x_test.reshape((-1,28,28,3)))
augmented_test = datagen.flow(x_test.reshape((-1,28,28,3)),y_test,batch_size=32)

# **Define the VGG16 model**


In [42]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32, 3))

# Freeze the VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

# Add layers for the classification task
model = Sequential()
model.add(keras.layers.experimental.preprocessing.Resizing(32, 32))
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy'
])

In [28]:
model.fit(augmented_train,epochs=10)

Epoch 1/10
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1507 - accuracy: 0.9502
Epoch 2/10
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1230 - accuracy: 0.9596
Epoch 3/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.1188 - accuracy: 0.9608
Epoch 4/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1106 - accuracy: 0.9633
Epoch 5/10
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1074 - accuracy: 0.9652
Epoch 6/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.1022 - accuracy: 0.9660
Epoch 7/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0984 - accuracy: 0.9663
Epoch 8/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0964 - accuracy: 0.9682
Epoch 9/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0941 - accuracy: 0.9690
Epoch 10/10
1875/1875 [==============================] - 41s 22m

In [43]:
model.summary()

ValueError: ignored

In [32]:
loss,accu = model.evaluate(augmented_test)
print(accu)

313/313 [==============================] - 6s 20ms/step - loss: 0.1067 - accuracy: 0.9659
0.9659000039100647
